# Suitability Analysis for a new Venue Location

[![open_in_colab][colab_badge]][colab_notebook_link]
<!-- [![open_in_binder][binder_badge]][binder_notebook_link] -->

[colab_badge]: https://colab.research.google.com/assets/colab-badge.svg
[colab_notebook_link]: https://colab.research.google.com/github/foursquare/fsq-studio-sdk-examples/blob/master/python-notebooks/09%20-%20Suitability_Analysis.ipynb
<!-- [binder_badge]: https://mybinder.org/badge_logo.svg
[binder_notebook_link]: https://mybinder.org/v2/gh/foursquare/fsq-studio-sdk-examples/master?urlpath=lab/tree/python-notebooks/09%20-%20Suitability_Analysis.ipynb -->

## Dependencies
This notebook uses the following dependencies:

* pandas
* numpy
* scikit-learn
* matplotlib
* h3
* catboost
* missingno

If running this notebook in Binder, these dependencies should already be installed. If running in Colab, the next cell will install these dependencies. In another environment, you'll need to make sure these dependencies are available by running the following `pip` command in a shell.

```
pip install pandas numpy scikit-learn seaborn matplotlib h3 catboost missingno
```

This notebook was originally tested with the following package versions, but likely works with a broad range of versions:

* pandas==1.3.3
* numpy==1.21.2
* scikit-learn==0.24.2
* scipy==1.7.1
* matplotlib==3.4.3
* h3==3.7.3
* catboost==0.26.1
* missingno==0.5.0

In [ ]:
# If in Colab, install this notebook's required dependencies
import sys
if "google.colab" in sys.modules:
    !pip install 'unfolded.map_sdk>=1.0' pandas numpy scikit-learn seaborn matplotlib h3 catboost missingno

## Imports

In [ ]:
from uuid import uuid4

import h3.api.numpy_int as h3
import matplotlib.pyplot as plt
import missingno as msno
import pandas as pd

%matplotlib inline
from catboost import CatBoostRegressor
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from unfolded.map_sdk import create_map

## Data Loading

In [ ]:
foot_traffic_url = 'https://actionengine-public.s3.us-east-2.amazonaws.com/common_df.csv'
# TODO: Save this DF without an index column
cities_url = 'https://actionengine-public.s3.us-east-2.amazonaws.com/cities.csv'

# TODO: Convert lon/lat columns to numeric here
foot_traffic_df = pd.read_csv(foot_traffic_url)
# TODO: Rename this `block_groups`?
cities_df = pd.read_csv(cities_url)

foot_traffic_df = foot_traffic_df.merge(cities_df, how='left', on=[
    "Neighbourhood Latitude", "Neighbourhood Longitude"
])

foot_traffic_df.head()

## Data Cleaning

In [ ]:
def compute_missing_values(df):
    """Compute missing values of table"""
    # Count of rows that are missing for each column
    missing_value_counts = df.isnull().sum()

    # Names of columns that have at least one missing row
    missing_cols = missing_value_counts[missing_value_counts > 0].index

    # Percent missing within each column
    missing_value_pct = (missing_value_counts / len(df) * 100).round(1)

    # Print how many columns have missing values
    print(
        f"Out of {len(df.columns)} columns, there are {len(missing_cols)} columns with missing values.")

    # Print table with information about missing columns
    missing_df = pd.DataFrame({
        'Missing Values': missing_value_counts[missing_cols],
        '% of Total Values': missing_value_pct[missing_cols],
    })
    print(missing_df)

# Look at missing values within foot_traffic_df
compute_missing_values(foot_traffic_df)

Use missingno to graphically show which columns have missing data:

In [ ]:
msno.matrix(foot_traffic_df)

There are many distinct values in the `'Venue Category'` colummn, which we can see by using `value_counts()`. Some of these categories have only one item in the data.

In [ ]:
foot_traffic_df['Venue Category'].value_counts()

Here we'll keep only the categories that have at least 10 venues. You can set your own minimum by changing `MIN_VENUES`.

In [ ]:
MIN_VENUES = 10
categories = foot_traffic_df['Venue Category'].value_counts()
category_names = categories[categories >= MIN_VENUES].index
foot_traffic_df = foot_traffic_df[foot_traffic_df['Venue Category'].isin(category_names)]

## Data Visualization

Here we'll create a map that clusters by districts with the largest population. Note that since the clustering happens within Foursquare Studio, the clusters change as you zoom in, allowing you to explore your data at various resolutions.

In [ ]:
population_neighbourhoods = create_map()
population_neighbourhoods

In [ ]:
neighbourhoods = foot_traffic_df.drop_duplicates(
    subset=["Neighbourhood Latitude", "Neighbourhood Longitude"]
)

# Create a persistent dataset ID that we can reference in both add_dataset and add_layer
dataset_id = str(uuid4())

population_neighbourhoods.add_dataset(
    id=dataset_id,
    label="population_neighbourhoods_NY",
    data=neighbourhoods,
    auto_create_layers=False,
)

population_neighbourhoods.add_layer(
    {
        "id": "population_neighbourhoods",
        "label": "population in NY",
        "type": "cluster",
        "data_id": dataset_id,
        "fields": {
                    "lat": "Neighbourhood Latitude",
                    "lng": "Neighbourhood Longitude",
        },
        "config": {
            "visual_channels": {
                "colorScale": "quantize",
                "colorField": {"name": "Total Population (Estimate)", "type": "real"}
            },
            "vis_config": {
                "colorRange": {
                    "colors": [
                        "#355C7D",
                        "#63617F",
                        "#916681",
                        "#D88185",
                        "#E8998D",
                        "#F8B195",
                    ]
                }, 
            },
        },
    }
)

population_neighbourhoods.set_view(longitude=-73.769652, latitude=40.710574, zoom=9)

This map that clusters by districts with the largest incomes.

In [ ]:
incomes_neighbourhoods = create_map()
incomes_neighbourhoods

In [ ]:
# Create a persistent dataset ID that we can reference in both add_dataset and add_layer
dataset_id = str(uuid4())

incomes_neighbourhoods.add_dataset(
    id=dataset_id,
    label="venues_NY",
    data=neighbourhoods,
    auto_create_layers=False,
)

incomes_neighbourhoods.add_layer(
    {
        "id": "incomes_neighbourhoods",
        "label": "incomes in NY by neighbourhoods",
        "type": "hexagon",
        "data_id": dataset_id,
        "fields": {
                    "lat": "Neighbourhood Latitude",
                    "lng": "Neighbourhood Longitude",
        },
        "config": {
            "visual_channels": {
                "colorScale": "quantize",
                "colorField": {"name": "Median Household Income(Estimate)", "type": "real"}
            },
            "vis_config": {
                "colorRange": {
                    "colors": [
                        "#006837",
                        "#31A354",
                        "#78C679",
                        "#ADDD8E",
                        "#D9F0A3",
                        "#FFFFCC",
                    ]
                }, 
            },
        },
    }
)

incomes_neighbourhoods.set_view(longitude=-73.769652, latitude=40.710574, zoom=9)

## Feature Engineering

Group data by district, then count the number of venues of each category for each district, and based on this, make a clustering of districts.

In [ ]:
districts_by_venue_cat = (
    foot_traffic_df.groupby(["District", "Venue Category"])["Venue Category"]
    .aggregate("count")
    .unstack()
)

# The groupby creates cells with NaN values for any district that didn't have the given type of venue
# Here we replace those NaN values with 0
districts_by_venue_cat = districts_by_venue_cat.fillna(0)

# Divide each cell by the total number of venues
# We sum across the rows and then use that number 
districts_by_venue_cat = districts_by_venue_cat.div(
    districts_by_venue_cat.sum(axis=1), axis=0
)

### Feature Scaling

Using the elbow method, we determine how many clusters we can divide the data

In [ ]:
# Elbow method for clustering
sum_of_squared_distances = []

K = range(1, 15)

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=0).fit(districts_by_venue_cat)
    sum_of_squared_distances.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(K, sum_of_squared_distances, "bx-")
plt.title("Elbow Method For Optimal k")

According to the graph above, the optimal value is 7

In [ ]:
kclusters = 7

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(districts_by_venue_cat)

districts_by_venue_cat["Label"] = kmeans.labels_

Merge the resulting cluster label back onto the main dataset `foot_traffic_df`

In [ ]:
foot_traffic_df = foot_traffic_df.merge(districts_by_venue_cat[["Label"]], left_on='District', right_index=True)

We will leave only those columns that are needed to build the model.

In [ ]:
final_df = foot_traffic_df[
    [
        "Neighbourhood Latitude",
        "Neighbourhood Longitude",
        "Venue Category",
        "Venue Latitude",
        "Venue Longitude",
        "County",
        "District",
        "Label",
        "Total Population (Estimate)",
        "Total Population (MarginOfError)",
        "Median Household Income(Estimate)",
        "Median Household Income(MarginOfError)",
        "raw_device_counts",
    ]
]

final_df

Visualize the principle of clustering 

In [ ]:
clustering_of_district = create_map()
clustering_of_district

In [ ]:
# Create a persistent dataset ID that we can reference in both add_dataset and add_layer
dataset_id = str(uuid4())

clustering_of_district.add_dataset(
    id=dataset_id,
    label="district clustering",
    data=final_df,
    auto_create_layers=False,
)

clustering_of_district.add_layer(
    {
        "id": "income_neighbourhoods",
        "label": "incomes in NY by neighbourhoods",
        "type": "point",
        "data_id": dataset_id,
        "fields": {
                    "lat": "Neighbourhood Latitude",
                    "lng": "Neighbourhood Longitude",
        },
        "config": {
            "visual_channels": {
                "colorScale": "quantize",
                "colorField": {"name": "Label", "type": "real"}
            },
            "vis_config": {
                "colorRange": {
                    "colors": [
                        "#B35806",
                        "#F1A340",
                        "#FEE0B6",
                        "#D8DAEB",
                        "#998EC3",
                        "#542788",
                    ]
                }
            },
        },
    }
)

clustering_of_district.set_view(longitude=-73.769652, latitude=40.710574, zoom=9)

## Data Preprocessing

In [ ]:
final_df.loc[:, "Venue Latitude"] = pd.to_numeric(final_df["Venue Latitude"])
X = final_df.drop(["raw_device_counts"], axis=1)
y = final_df[["raw_device_counts"]]

## Data Splitting

We split the data into training, validation and test sets

In [ ]:
# dividing training data into test, validation and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=1
)

# define categorical features
cat_features = list(X.select_dtypes('object').columns)

As we mentioned before, two important columns contain empty values, for this reason fill them with median value.

In [ ]:
empty_values = [
    "Median Household Income(Estimate)",
    "Median Household Income(MarginOfError)",
]

median_values = [
    X_train["Median Household Income(Estimate)"].median(axis=0),
    X_train["Median Household Income(MarginOfError)"].median(axis=0),
]


def fill_empty_values(df, columns, median_values):
    for col, med in zip(columns, median_values):
        df[col].fillna(
            med, axis=0, inplace=True,
        )


fill_empty_values(X_train, empty_values, median_values)
fill_empty_values(X_val, empty_values, median_values)
fill_empty_values(X_test, empty_values, median_values)

## Foot Traffic Prediction Model

### Building

We use here CatBoost (new open-source machine learning algorithm, developed in 2017 by Yandex company), because it offers a way of handling categorical data. Also builds upon the theory of decision trees and gradient boosting and thus through greedy search create a strong competitive predictive model.

In [ ]:
model = CatBoostRegressor(
    iterations=1000, depth=4, learning_rate=0.1, loss_function="RMSE"
)

model.fit(
    X_train, y_train, cat_features=cat_features, eval_set=(X_val, y_val), plot=True
)

pred = model.predict(X_test)

### H3 grid

Divide all site data with a hexagonal grid

In [ ]:
final_df["h3_idx"] = final_df.apply(
    lambda row: h3.geo_to_h3(row["Venue Latitude"], row["Venue Longitude"], 10), axis=1
)

In [ ]:
# Use a lambda function to pick the _first_ mode in case there are multiple modes
mode_fn = lambda x: pd.Series.mode(x)[0]

grouped_by_h3 = final_df.groupby(["h3_idx"]).agg(
    {
        "Neighbourhood Latitude": mode_fn,
        "Neighbourhood Longitude": mode_fn,
        "County": mode_fn,
        "District": mode_fn,
        "Label": mode_fn,
        "Total Population (Estimate)": "mean",
        "Total Population (MarginOfError)": "mean",
        "Median Household Income(Estimate)": "mean",
        "Median Household Income(MarginOfError)": "mean",
    }
)
grouped_by_h3[['Venue Latitude', 'Venue Longitude']] = list(pd.Series(grouped_by_h3.index).apply(h3.h3_to_geo))

In [ ]:
X_test_H3 = grouped_by_h3.reset_index()

X_test_H3["Venue Category"] = "Pizza Place"

X_test_H3 = X_test_H3[
    [
        "h3_idx",
        "Neighbourhood Latitude",
        "Neighbourhood Longitude",
        "Venue Category",
        "Venue Latitude",
        "Venue Longitude",
        "County",
        "District",
        "Label",
        "Total Population (Estimate)",
        "Total Population (MarginOfError)",
        "Median Household Income(Estimate)",
        "Median Household Income(MarginOfError)",
    ]
]

### Prediction

In [ ]:
predicted_data = X_test_H3.copy()

X_test_H3 = X_test_H3.drop(["h3_idx"], axis=1)

predicted_devices_counts_by_h3 = model.predict(X_test_H3)

predicted_data["raw_device_count"] = predicted_devices_counts_by_h3
predicted_data["h3_idx"] = predicted_data["h3_idx"].apply(lambda x: format(x, 'x'))

This map shows the subdivision of a city using a H3 grid.

In [ ]:
raw_devices_data_by_h3 = create_map()
raw_devices_data_by_h3

In [ ]:
# Create a persistent dataset ID that we can reference in both add_dataset and add_layer
dataset_id = str(uuid4())

raw_devices_data_by_h3.add_dataset(
    id=dataset_id,
    label="venues_NY",
    data=predicted_data,
    auto_create_layers=False,
)

raw_devices_data_by_h3.add_layer(
    {
        "id": "raw_devices_data_by_h3",
        "label": "h3_idx",
        "type": "h3",
        "data_id": dataset_id,
        "fields": {
            "hex_id": "h3_idx",
        },
        "config": {
            "visual_channels": {
                "colorField": {"name": "raw_device_count", "type": "real"}
            },
            "vis_config": {
                "opacity": 0.8,
                "color_range": {
                    "name": "ColorBrewer Reds-6",
                    "type": "singlehue",
                    "category": "ColorBrewer",
                    "colors": [
                        "#fee5d9",
                        "#fcbba1",
                        "#fc9272",
                        "#fb6a4a",
                        "#de2d26",
                        "#a50f15",
                    ],
                }
            },
        },
    }
)

raw_devices_data_by_h3.set_view(longitude=-73.769652, latitude=40.710574, zoom=9)

### Visualize results

This map shows predicted count of devices by districts of NYC

In [ ]:
venues_NY = create_map()
venues_NY

In [ ]:
# Create a persistent dataset ID that we can reference in both add_dataset and add_layer
dataset_id = str(uuid4())

venues_NY.add_dataset(
    id=dataset_id,
    label="venues_NY",
    data=predicted_data,
    auto_create_layers=False,
)

venues_NY.add_layer(
    {
        "id": "foot_traffic_data_by_venues",
        "label": "Predicted Foot Traffic By raw_device_count",
        "type": "point",
        "data_id": dataset_id,
        "fields": {
            "lat": "Venue Latitude",
            "lng": "Venue Longitude"
        },
        "config": {
            "visual_channels": {
                "color_scale": "quantile",
                "colorField": {"name": "raw_device_count", "type": "real"}
            },
            "vis_config": {
                "opacity": 0.08,
                "colorRange": {
                    "colors": [
                        "#2B1E3E",
                        "#343D5E",
                        "#4F777E",
                        "#709E87",
                        "#99BE95",
                        "#D6DEBF",
                    ],
                },
            },
        },
    }
)

venues_NY.set_view(longitude=-73.769652, latitude=40.710574, zoom=9)

Visualize the results with a heatmap, we can see the areas with the highest foot traffic

In [ ]:
venues_NY_heatmap = create_map()
venues_NY_heatmap

In [ ]:
# Create a persistent dataset ID that we can reference in both add_dataset and add_layer
dataset_id = str(uuid4())

venues_NY_heatmap.add_dataset(
    id=dataset_id,
    label="venues_NY",
    data=predicted_data,
    auto_create_layers=False,
)

venues_NY_heatmap.add_layer(
    {
        "id": "foot_traffic_data_by_venues",
        "label": "Predicted Foot Traffic By raw_device_count",
        "type": "heatmap",
        "data_id": dataset_id,
        "fields": {
            "lat": "Venue Latitude",
            "lng": "Venue Longitude"
        },
        "config": {
            "visual_channels": {
                "weightField": {"name": "raw_device_count", "type": "real"},
                "weightScale": "linear",
            },
            "vis_config": {
                "opacity": 0.8,
                "colorRange": {
                    "name": "Global Warming",
                    "type": "sequential",
                    "category": "Uber",
                    "colors": [
                        "#5A1846",
                        "#900C3F",
                        "#C70039",
                        "#E3611C",
                        "#F1920E",
                        "#FFC300",
                    ],
                },
                "radius": 20,
                "intensity": 0.7,
                "threshold": 0.189,
            },
        },
    }
)

venues_NY_heatmap.set_view(longitude=-73.769652, latitude=40.710574, zoom=9)